In [6]:
# Python standard library packages 
import sys
import os

# 3rd party packages 
# import pandas as pd
# !pip install python-dotenv
import dotenv
import sqlalchemy as sa#   

# Local development packages
sys.path.append(".")
import trapdata
from trapdata.db import get_session_class

%load_ext autoreload
%autoreload 2

dotenv.load_dotenv()

db_path = os.getenv("DATABASE_URL")
Session = get_session_class(db_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
session = Session()

In [3]:
session.execute(sa.select(trapdata.DetectedObject.id).limit(10)).scalars().all()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [9]:
query = sa.select(trapdata.DetectedObject.specific_label)
species = [r[0] for r in session.execute(query).unique().all()]
species

['Argyrotaenia quadrifasciana',
 'Martyringa latipennis',
 'Cenopis directana',
 None,
 'Herpetogramma pertextalis',
 'Aglossa cuprina',
 'Microcrambus minor',
 'Heterocampa biundata',
 'Choristoneura parallela',
 'Sibine stimulea',
 'Crambidia pallida',
 'Eudonia heterosalis',
 'Oreta rosea',
 'Eulithis diversilineata',
 'Lophocampa caryae',
 'Eumacaria madopata',
 'Epelis truncataria',
 'Grammia virguncula',
 'Dyseriocrania griseocapitella',
 'Disclisioprocta stellata',
 'Haploa confusa',
 'Lampronia corticella',
 'Archips argyrospila',
 'Lygropia rivulalis',
 'Pyrausta insequalis',
 'Parasemia plantaginis',
 'Anania hortulata',
 'Polygrammate hebraeicum',
 'Alsophila pometaria',
 'Dichomeris isa',
 'Argyrotaenia alisellana',
 'Pseudosciaphila duplex',
 'Argyrotaenia quercifoliana',
 'Elophila ekthlipsis',
 'Dioryctria disclusa',
 'Orthofidonia flavivenata',
 'Petrophila canadensis',
 'Coleophora limosipennella',
 'Monopis marginistrigella',
 'Endothenia hebesana',
 'Crambus laqueate

In [24]:
def get_detections(species):
    for sp in species:
        query = sa.select(
            trapdata.DetectedObject.path, 
            trapdata.DetectedObject.specific_label_score
            ).where(trapdata.DetectedObject.specific_label == sp)
        rows = [o for o in session.execute(query).all()]
        yield (sp, rows)

next(get_detections(species))

('Argyrotaenia quadrifasciana',
 [('/Users/michael/Projects/AMI/USERDATA/crops/c5b016ae835e743cbb25e38157d0e4c9.jpg', 0.3960445523262024),
  ('/Users/michael/Projects/AMI/USERDATA/crops/b356bfdd42f584b4ff7e5d5f89629c4a.jpg', 0.7776317000389099),
  ('/Users/michael/Projects/AMI/USERDATA/crops/bcfc6fec69b447730765635323e0cec6.jpg', 0.4307246804237366),
  ('/Users/michael/Projects/AMI/USERDATA/crops/fc3c9ccaefd6f27bde1531e005e894aa.jpg', 0.45581212639808655),
  ('/Users/michael/Projects/AMI/USERDATA/crops/1f36e79256c9960d89bd99f696c03bbe.jpg', 0.49879539012908936),
  ('/Users/michael/Projects/AMI/USERDATA/crops/76afd4271d864e2c82b55e699394c6cd.jpg', 0.315772145986557),
  ('/Users/michael/Projects/AMI/USERDATA/crops/1346681ffa512e9161078e3cd8c6df31.jpg', 0.8750147223472595),
  ('/Users/michael/Projects/AMI/USERDATA/crops/46d508c818878e1bf023d6b1aff41ffd.jpg', 0.9404839277267456),
  ('/Users/michael/Projects/AMI/USERDATA/crops/99f5545ab187ce221113d11380e3b6c7.jpg', 0.6836795806884766),
  ('

In [29]:
def make_fragment(species, rows):
    images = "".join([f"<img src='{r[0]}' title='{r[1]}'>"  for r in rows])
    html = f"""
    <h3>{species}</h3>
    {images}
    """
    return html

In [30]:
styles = """
<style>
h3 { font-family: sans-serif; }
img { margin: 4px; }
</style>
"""

with open("detections.html", "w") as f:
    f.write(styles)
    for sp, rows in get_detections(species):
        html = make_fragment(sp, rows)
        f.write(html)